# Price Optimization for Hotels

In [ ]:
pip install --upgrade pip

# Getting snowflake related packages installed and imported

In [ ]:
pip install "snowflake-connector-python[pandas]"

In [ ]:
pip install --upgrade --q snowflake-snowpark-python==1.9.0

In [ ]:
pip install fosforio

In [ ]:
pip install fosforml

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install scikit-learn

In [ ]:
pip install xgboost

In [ ]:
pip install seaborn

In [ ]:
pip install python-dateutil

In [ ]:
pip install tqdm

In [ ]:
pip install holidays

# Restart kernal and continue

# Importing packages

In [1]:
from snowflake.snowpark import Session

ModuleNotFoundError: No module named 'cryptography.__about__'

In [2]:
from snowflake.connector.pandas_tools import write_pandas

ModuleNotFoundError: No module named 'cryptography.__about__'

In [3]:
from fosforio import snowflake

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import numpy as np
import pandas as pd

In [6]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error

Matplotlib created a temporary cache directory at /tmp/matplotlib-3c0sxlnw because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [7]:
from joblib import dump, load
import requests
from tqdm import tqdm
import time
from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime

# Getting data from snowflake

In [10]:
# To get snowflake connection object with a default snowflake connection created by the user, if available.
snowflake.get_connection("TTH_REV_OPT_CXN")

Connection details fetched: {'params': {'READER': {'connectionId': '73423620-7e1f-47a2-b9c5-b37408757323', 'connectionSources': {'sourceId': '210d73c1-4da7-487d-8a5c-05c8d2025269', 'connectionType': 'SNOWFLAKE', 'sourceName': 'RDBMS', 'enable': 1, 'displayName': 'SNOWFLAKE', 'displayOrder': 5.0}, 'createdBy': 'ms.akshaya@fosfor.com', 'createdOn': '2024-05-24T10:53:33.084+00:00', 'updatedOn': '2024-06-06T07:18:09.475+00:00', 'name': 'TTH_REV_OPT_CXN', 'autosyncable': False, 'autopublishable': False, 'frequency': None, 'assetConfigurations': [], 'publishableNodes': [], 'autoPublishableNodes': None, 'isPrivate': False, 'isExpired': False, 'autoSyncFrequency': None, 'application': None, 'sourceDataRefreshFrequency': None, 'defaultDb': 'TTH_DB', 'defaultSchema': 'TTH_REV_OPT_SCHEMA', 'recommendedPrivileges': {'DATA_MODULE': {'usage_on_schema': True, 'usage_on_database': True, 'create_table_on_schema': True, 'select_insert_update_truncate_delete_ref_on_tables': True}, 'INSIGHT_MODULE': {}, '

In [9]:
df = snowflake.get_dataframe("BOOKINGS_TRANSFORMED")
df['ADR']= round(df['ADR'])

Connection details fetched: {'params': {'READER': {'schema': 'TTH_REV_OPT_SCHEMA', 'cloudPlatform': 'gcp', 'role': '"AKSHAYA"', 'clientId': None, 'tokenUrl': None, 'scopeUrl': None, 'type': 'RDBMS', 'wareHouse': 'FOSFOR_SOLUTIONS_WH', 'accountId': 'ug94937', 'privateKey': None, 'password': 'Password@2023', 'database': 'TTH_DB', 'tables': 'BOOKINGS_TRANSFORMED', 'sub_type': 'SNOWFLAKE', 'subType': 'SNOWFLAKE', 'clientSecret': None, 'authenticationType': 'dbPassword', 'rowCount': 10, 'region': 'us-east4', 'user': 'Akshaya', 'applicationName': 'FOSFOR', 'passPhrase': None}}, 'url': 'http://connectors-backend-service/connectors/v1/connectors/metadata', 'access': True, 'errorMsg': None, 'values': {}}
Ex: No module named 'cryptography.__about__'
Exception occurred in reading data_frame from snowflake connection: Exception occurred in creating snowflake connection: None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
#download star schema tables
meal = 
customer = 
channel = 
room = 

In [ ]:
df.shape

In [ ]:
df.info() 

In [ ]:
df.columns = [x.lower() for x in df.columns]

In [ ]:
df.info()

In [ ]:
sns.boxplot(data=df, x="reserved_room_type", y="adr", hue= 'is_canceled', color=".8", linecolor="#137", linewidth=.75)

In [ ]:
quartiles_data = df.groupby(['hotel', 'reserved_room_type'])['adr'].agg(
    adr_mean='mean',
    adr_median='median',
    adr_q01=lambda x: x.quantile(0.10),
    adr_q1=lambda x: x.quantile(0.25),
    adr_q2=lambda x: x.quantile(0.50),
    adr_q3=lambda x: x.quantile(0.75),
    adr_q4=lambda x: x.quantile(1.00)
).reset_index()
print(quartiles_data)

In [ ]:
# Merge the quartiles data with the original data
merged_data = pd.merge(df, quartiles_data, on=['hotel', 'reserved_room_type'])

print("\nMerged Data:")
print(merged_data)

In [ ]:
filtered_data = merged_data[(merged_data['adr'] > merged_data['adr_q01']) & 
                            (merged_data['adr'] < merged_data['adr_q4'])]

print("\nFiltered Data:")
print(filtered_data)

In [ ]:
filtered_data.info()

In [ ]:
# renaming some column names
filtered_data.rename(columns = {'arrival_date_transformed':'arrival_date', 
                                'reservation_status_date_transformed':'reservation_date',
                                'arrival_date_year':'year','arrival_date_week_number': 'week', 'arrival_date_day_of_month': 'dom', 
                                'total_stay_nights': 'total_rns',
                                'stays_in_weekend_nights':'weekend_nights', 'stays_in_week_nights': 'week_nights',
                                'reserved_room_type': 'room_type', 'customer_type':'cust_type', 'ADR':'adr'}, inplace = True)


In [ ]:
filtered_data.info()

In [ ]:
sns.boxplot(data=filtered_data, x="room_type", y="adr", hue= 'is_canceled', color=".8", linecolor="#137", linewidth=.75)

In [ ]:
sns.boxplot(data=filtered_data, x="room_type", y="adr", hue= 'market_segment', 
            fill=False, gap=.1, linecolor="#137", linewidth=.75, color= 'b')

In [ ]:
# Create a combined column for the plot
filtered_data['Group'] = filtered_data['hotel'] + ' - ' + filtered_data['room_type']

# Set the size of the plot
plt.figure(figsize=(8, 4))

# Create boxplot
sns.boxplot(x='Group', y='adr', data=filtered_data)

# Customize the plot
plt.title('Boxplot of ADR by Hotel and Reserved Room Type')
plt.xlabel('Group')
plt.ylabel('adr')

# Show plot
plt.show()


In [ ]:
# Create a combined column for the plot
filtered_data['Group3'] = filtered_data['room_type'] + ' - ' + filtered_data['reservation_status']

# Set the size of the plot
plt.figure(figsize=(12, 4))

# Create boxplot
sns.boxplot(x='Group3', y='adr', data=filtered_data)

# Customize the plot
plt.title('Boxplot of ADR by CANCELED and Reserved Room Type')
plt.xlabel('Group3')
plt.ylabel('adr')

# Show plot
plt.show()

# EDA

In [ ]:
# Removing cancelled transactions to model based on transactions that customers preferred to actually complete
df0 = filtered_data[(filtered_data['is_canceled'] == 0) & (filtered_data['reservation_status'] !='No-Show')] 
df0

In [ ]:
#Understanding what market segments need to be considered
df0.groupby(['hotel','room_type','market_segment']).agg({'adr':'mean','reservation_date':'count'})

#We will filter for room _types A,D,E and will filter OUT "Complementary" market_segment. retain both City and resort hotel types.
##"For our pricing model, we need to count the frequency of each price point

##Moreover, Hotels have different room types with different prices. To create our pricing model, we need to optimize the room type level. We are interested in the room type reserved as this captures the price the guest was willing to pay for that room type. It is common for hotels to overbook the lower room types and give free upgrades to higher room types. Therefore, any pricing analysis must be done at the reserved room type and not the stayed room type.

##Unfortunately, there is not much description of the reserved room type, but by looking at the price points, we can infer that A &B are the lowest price points. The rest of the room types command a higher ADR, which means they likely are upgraded rooms. In another workbook, we will explore how to price those premium rooms, but for the moment, let's focus on room types A & D, E.

#filter data to only Direct and Online TA and room typ A&B
##Create individual datasets for City, Resort, room types: A,D,E

In [ ]:
#data1 = df0[(df0['market_segment'] != 'Complementary') & (df0['market_segment'] != 'Corporate') 
#           & (df0['market_segment'] !='Aviation')]
data1 = df0[~df0['market_segment'].isin(['Complementary', 'Corporate', 'Aviation'])]
#data1 = df0[(df0['room_type'] == 'A') |(df0['room_type'] == 'D') | (df0['room_type'] == 'E')]
data1.room_type.value_counts()

In [ ]:
data1.market_segment.value_counts()

In [ ]:
#Understanding what market segments need to be considered
data1.groupby(['hotel','room_type','market_segment']).agg({'adr':'mean','reservation_date':'count'})

In [ ]:
data1.info()

In [ ]:
# checking num dates
data1.groupby(['arrival_date']).agg({'adr':'mean','reservation_date':'count'})

In [ ]:
data1.info()

In [ ]:
data1.head(2)

In [ ]:
# drop unrequired columns
data2 = data1.drop(['tally_days', 'deposit_type', 'assigned_room_type','is_canceled','adults', 'children', 'babies', 
                   'avg_rooms_per_night',
                   'meal', 'country', 'distribution_channel', 'previous_bookings_not_canceled','deposit_type',
                   'days_in_waiting_list', 'reservation_status_date', 'previous_cancellations','expected_arrival_date',
                   'lead_time','total_room_nights','adr_mean','adr_median','adr_q01','adr_q1' ,'adr_q2','adr_q3','adr_q4',                       
                   'Group','Group3'], axis =1)
data2.info()

In [ ]:
# renaming some column names
# data1.rename(columns = {'ARRIVAL_DATE_TRANSFORMED':'arrival_date', 'RESERVATION_STATUS_DATE_TRANSFORMED':'reservation_date',
#                        'HOTEL':'hotel', 'ARRIVAL_DATE_YEAR':'year','MONTH':'month', 
#                       'ARRIVAL_DATE_WEEK_NUMBER': 'week', 'ARRIVAL_DATE_DAY_OF_MONTH': 'dom', 
#                       'RESERVATION_STATUS': 'reservation_status', 'TOTAL_STAY_NIGHTS': 'total_rns',
#                       'STAYS_IN_WEEKEND_NIGHTS':'weekend_nights', 
#                       'STAYS_IN_WEEK_NIGHTS': 'week_nights','TOTAL_GUESTS': 'total_guests','MARKET_SEGMENT':'market_segment', 
#                       'RESERVED_ROOM_TYPE': 'room_type', 'CUSTOMER_TYPE':'cust_type', 'ADR':'adr'}, inplace = True) ##

In [ ]:
# checking num dates
data2.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
# Creating separate tables for Optimizing models
ca = data2[(data2['hotel'] == 'City Hotel') & (data2['room_type'] == 'A')]
cd = data2[(data2['hotel'] == 'City Hotel') & (data2['room_type'] == 'D')]
ce = data2[(data2['hotel'] == 'City Hotel') & (data2['room_type'] == 'E')]
ra = data2[(data2['hotel'] == 'Resort Hotel') & (data2['room_type'] == 'A')]
rd = data2[(data2['hotel'] == 'Resort Hotel') & (data2['room_type'] == 'D')]
re = data2[(data2['hotel'] == 'Resort Hotel') & (data2['room_type'] == 'E')]

In [ ]:
ca.info()

In [ ]:
# checking num dates
ca.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
cd.info()

In [ ]:
# checking num dates
cd.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
ce.info()

In [ ]:
# checking num dates
ce.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
ra.info()

In [ ]:
# checking num dates
ra.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
rd.info()

In [ ]:
# checking num dates
rd.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

In [ ]:
re.info()

In [ ]:
# checking num dates
re.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

# Data transformation to disaggregate dates
# For Full data

In [ ]:
data_to_transform = data2[['hotel','room_type','arrival_date','total_rns','adr']]
data_to_transform.info()

In [ ]:
# Create a new dataframe to store the data by stay date
expanded_df = pd.DataFrame()

In [ ]:
# Iterate over each row in the expected arrival date
##Need to convert with hotel, room type columns
for _, row in data_to_transform.iterrows():
    # Get the number of stay dates for the current booking
    num_stay_dates = row['total_rns']
    
    #Create a row for each stay date
    expanded_booking = pd.DataFrame({\
        'hotel': row['hotel'],
        'room_type' : row ['room_type'],                             
        'arrival_date': pd.date_range(start=row['arrival_date'], periods=num_stay_dates),
        'total_rns': 1,
        'adr': row['adr']
    })                                
    # Append the stay date information to the new dataframe
    expanded_df = pd.concat([expanded_df, expanded_booking], ignore_index=True)  

In [ ]:
# Sort the final dataframe by date
expanded_df = expanded_df.sort_values('adr')
expanded_df = expanded_df.reset_index(drop=True)
expanded_df

## Building seasonality

In [ ]:
import holidays
holiday_dates = holidays.CountryHoliday('PT', years=[2020,2021,2022,2023])
holidays = {
    expected_arrival_date: name
    for expected_arrival_date, name in holiday_dates.items()
    if name in ['Ano Novo', 'Páscoa', 'Dia de Natal']
}

In [ ]:
# rename holiday columns
expanded_df = expanded_df.rename({'Ano Novo':'new_year','Páscoa':'easter','Dia de Natal':'christmas'},axis=1)

holidays = {datetime.date(2023, 1, 1): 'new_year',
 datetime.date(2023, 3, 27): 'easter',
 datetime.date(2023, 12, 25): 'christmas',
 datetime.date(2022, 1, 1): 'new_year',
 datetime.date(2022, 4, 16): 'easter',
 datetime.date(2022, 12, 25): 'christmas',
 datetime.date(2021, 1, 1): 'new_year',
 datetime.date(2021, 4, 5): 'easter',
 datetime.date(2021, 12, 25): 'christmas',
 datetime.date(2020, 1, 1): 'new_year',
 datetime.date(2020, 4, 5): 'easter',
 datetime.date(2020, 12, 25): 'christmas'}

# updated code

In [ ]:
def generate_holiday_dates(start_year, end_year):
    holidays = {}
    for year in range(start_year, end_year + 1):
        holidays[datetime.date(year, 1, 1)] = 'new_year'
        easter_date = pd.to_datetime(calendar().easter(year)).date()
        holidays[easter_date] = 'easter'
        holidays[datetime.date(year, 12, 25)] = 'christmas'
    return holidays

generate_holiday_dates(2020, 2023)

In [ ]:
# Define pre and post ranges for each holiday
pre_range_offset = {'new_year': relativedelta(days=-1),
                    'easter': relativedelta(days=-2),
                    'christmas': relativedelta(days=-3)}

post_range_offset = {'new_year': relativedelta(days=1),
                     'easter': relativedelta(days=2),
                     'christmas': relativedelta(days=3)}

In [ ]:
# Create new columns for each holiday
for holiday in holidays.values():
    expanded_df[holiday] = 0
 
 # Set the holiday columns to 1 for matching dates
for arrival_date, name in holidays.items():
    expanded_df.loc[expanded_df['arrival_date'] == arrival_date, name] = 1

    # Set the holiday columns to 1 for pre and post dates
    pre_offset = pre_range_offset.get(name)
    if pre_offset:
        pre_date = pd.to_datetime(arrival_date) + pre_offset
        expanded_df.loc[expanded_df['arrival_date'] == pre_date.strftime('%Y-%m-%d'), name] = 1

    post_offset = post_range_offset.get(name)
    if post_offset:
        post_date = pd.to_datetime(arrival_date) + post_offset
        expanded_df.loc[expanded_df['arrival_date'] == post_date.strftime('%Y-%m-%d'), name] = 1

In [ ]:
##check
expanded_df.head()

In [ ]:
# Add dow, month to data
expanded_df['dow'] = expanded_df.arrival_date.dt.strftime('%A')
expanded_df['month'] = expanded_df.arrival_date.dt.strftime('%B')

In [ ]:
##check
expanded_df.head()

In [ ]:
##PRICING MODEL 1 (WITHOUT HOLIDAYS)
#Remove holidays for our main model
non_holidays = expanded_df[expanded_df[['new_year', 'easter', 'christmas']].sum(axis=1) == 0]
non_holidays.head()

In [ ]:
##Our optimization will be done at the DOW and Month Level. As a reminder, we are trying to model the relationship between Price and Demand so that we can answer the below question: if I charge X , how many rooms will I sell? To begin with, we need to get the demand levels for each DOW and month as well as create a table counting how many times each price point appears for each DOW and month.

##ADD HOTEL, ROOM TYPE TO GROUPINGS
daily_rns = non_holidays.groupby(['arrival_date','dow','month']).agg({'total_rns':'sum'}).reset_index() # get total stays per day
daily_rns = daily_rns.groupby(['dow','month']).agg({'total_rns':['sum','mean','median']}).reset_index() # get Rns metrics by Dow & Month
daily_rns.columns = ['_'.join(col) for col in daily_rns.columns] #remove multi level column
daily_rns

In [ ]:
##Counting rate frequency
##In the next step, we will count how often each adr (price the guest paid) appears for each DOW and month.
# create rate frequency table
adr_frequency = non_holidays.groupby(['dow','month','adr']).agg({'total_rns':'sum'})
adr_frequency.reset_index(inplace=True)
adr_frequency.head(10)

# Data transformation to dis-aggregate dates- 
# For City Hotel room type A

In [ ]:
ca_data_to_transform = ca[['hotel','room_type','arrival_date','total_rns','adr']]
ca_data_to_transform.info()

In [ ]:
# Create a new dataframe to store the data by stay date
ca_expanded_df = pd.DataFrame()

In [ ]:
# Iterate over each row in the expected arrival date
##Need to convert with hotel, room type columns
for _, row in ca_data_to_transform.iterrows():
    # Get the number of stay dates for the current booking
    num_stay_dates = row['total_rns']
    
    #Create a row for each stay date
    ca_expanded_booking = pd.DataFrame({
        'hotel': row['hotel'],
        'room_type' : row ['room_type'], 
        'arrival_date': pd.date_range(start=row['arrival_date'], periods=num_stay_dates),
        'total_rns': 1,
        'adr': row['adr'] 
    })
    # Append the stay date information to the new dataframe
    ca_expanded_df = pd.concat([ca_expanded_df, ca_expanded_booking], ignore_index=True)  

In [ ]:
ca_expanded_df.info()

In [ ]:
ca_expanded_df.head(10)

In [ ]:
# Sort the final dataframe by date
ca_expanded_df = ca_expanded_df.sort_values('arrival_date')
ca_expanded_df = ca_expanded_df.reset_index(drop=True)
ca_expanded_df

In [ ]:
ca_expanded_df.info()